# Que vamos a hacer:
### 3 Sistemas de recomendación
- Afinidad a Marca por artistas
- Afinidad a Marca por géneros
- Afinidad a Marca por subgéneros

- Tenemos que tener en cuenta que el usuario puede que no coincida con nada
- Que los rankings pueden ser de distinta longitud (uno mas largo que el otro)
- Que el numero de apariciones no podemos usarlo, debemos usar como peso la posicion del ranking que tenga, pues el numero de apariciones setea un peso distinto e injusto (cantidad de muestras en marca contra solo uno)

In [1]:
import sys
sys.path.append("../")
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np
import os
from supabase import create_client, Client
from dotenv import load_dotenv
# Cargar variables de entorno
load_dotenv()

# Configuración de Supabase
url = os.getenv("project_url")
key = os.getenv("browser_safe_key")
supabase: Client = create_client(url, key)

# Afinidad a Marca por artistas

In [2]:
ranking_zara = supabase.table("artists_ranking").select("artist_name","number_of_appearances").eq("brand_id",2).order("number_of_appearances",desc=True).execute().data
artistas = []
apariciones = []
for ranking in ranking_zara:
    artistas.append(ranking["artist_name"])
    apariciones.append(ranking["number_of_appearances"])

df_zara = pd.DataFrame({
    "artista" : artistas,
    "apariciones" : apariciones
})

df_zara.index = df_zara.index + 1
df_zara.reset_index(inplace=True)
df_zara.columns = ["ranking","artista","apariciones"]
df_zara.drop(columns="apariciones",inplace=True)
df_zara["artista"] = df_zara["artista"].astype(str)

In [3]:
ranking_user = supabase.table("user_artists_ranking").select("artist_name","number_of_appearances").eq("user_id","gonzaloruiperez").order("number_of_appearances",desc=True).execute().data
artistas = []
apariciones = []
for ranking in ranking_user:
    artistas.append(ranking["artist_name"])
    apariciones.append(ranking["number_of_appearances"])

df_user = pd.DataFrame({
    "artista" : artistas,
    "apariciones" : apariciones
})

df_user.index = df_user.index + 1
df_user.reset_index(inplace=True)
df_user.columns = ["ranking","artista","apariciones"]
df_user.drop(columns="apariciones",inplace=True)
df_user["artista"] = df_user["artista"].astype(str)

# 1 Buscamos los artistas comunes
Dejando unicamente aquellos que coinciden con su posición en cada ranking!

In [4]:
# Obtener los artistas en común
artistas_comunes = set(df_user['artista']).intersection(set(df_zara['artista']))
df_user_filtrado = df_user[df_user['artista'].isin(artistas_comunes)]
df_zara_filtrado = df_zara[df_zara['artista'].isin(artistas_comunes)]

df_user_filtrado.reset_index(drop=True, inplace=True)
df_zara_filtrado.reset_index(drop=True, inplace=True)

# 2 Asignamos pesos según su posición en el ranking 

- Fórmula utilizada
$$
peso = \frac{1}{\text{posición}}
$$

No buscamos complejidad, buscamos que funcione de momento


In [5]:
# Asignar pesos en base a la posición en el ranking
df_user_filtrado["peso"] = 1 / df_user_filtrado["ranking"]
df_zara_filtrado["peso"] = 1 / df_zara_filtrado["ranking"]

# 3 Normalizamos los pesos
- Dado que la longitud y ranking es distinto, debemos normalizar los pesos a la misma escala
- De esta forma nos aseguramos de que el cálculo de la afinidad es justo
- Fórmula utilizada

$$
peso\_normalizado = \frac{peso\_original}{\sum peso\_original}
$$

- Ahora los pesos están a la misma escala

### Operador /=
- Es lo mismo que
```python
    df_user_filtrado["peso"] = df_user_filtrado["peso"] / df_user_filtrado["peso"].sum()
```

- Esto garantiza que todos los pesos sumen 1, permitiendo comparaciones justas entre rankings de distinto tamaño.
- Normalizar los pesos evita que rankings más largos o cortos afecten injustamente el cálculo de afinidad.


In [6]:
# Normalizar los pesos para evitar sesgos por diferencias de tamaño
df_user_filtrado["peso"] /= df_user_filtrado["peso"].sum()
df_zara_filtrado["peso"] /= df_zara_filtrado["peso"].sum()

# 4 Poner como index artista y peso se queda como columna

In [7]:
df_user_filtrado = df_user_filtrado.set_index("artista")[["peso"]]
df_zara_filtrado = df_zara_filtrado.set_index("artista")[["peso"]]

# 5 Renombrar columnas

In [8]:
df_user_filtrado.rename(columns={"peso":"peso_user"},inplace=True)
df_zara_filtrado.rename(columns={"peso":"peso_marca"},inplace=True)

# 6 Fusionar los DataFrames en uno solo alineado por artista

In [9]:
union = df_user_filtrado.merge(df_zara_filtrado,left_index=True,right_index=True)
union.head()

,peso_user,peso_marca
artista,,
dave rodgers,0.150431,0.000002
Hardwell,0.075215,0.000236
David Guetta,0.050144,0.033783
Max Coveri,0.037608,0.000004
Bizarrap,0.030086,0.000704


# 7 Calcular distancia euclidiana
- Con pdist calculamos distancia euclidiana y devuelve la matriz de distancias compacta
- Con squareform convertimos la matriz compacta en una matriz cuadrada de distancias

In [10]:
from scipy.spatial.distance import pdist, squareform
import numpy as np

distancia = squareform(pdist(union,metric="mahalanobis"))

# 8 Convertir la distancia en Similitud
- Convertimos a un rango de 0,1 la distancia
- Quitamos valores negativos
- hacemos que una menor distancia signifique mayor similitud

In [11]:
similitud = 1 / (1 + distancia)

# 9 Extraer afinidad a la marca

In [12]:
afinidad = similitud[0,1] * 100

In [159]:
afinidad

np.float64(7.69884742811001)

# 10 Ver resultados según distintas métricas
- cosine: 99.999%
- correlation: 100%
- canberra: 43,21%
- euclidean: 93,0045%
- mahalanobis: 7,69%

# No continuamos
- Debido a los resultados obtenidos, pasamos a otro modelo